# Filtado de mensajes spam

## Descripción del problema real

La recepción de publicidad no deseada a traves mensajes de texto usando SMS (Short Message Service) es un problema que afecta a muchos usuarios de teléfonos móviles. El problema radica en que los usuarios deben pagar por los mesajes recibidos, y por este motivo resulta muy importante que las compañías prestadoras del servicio puedan filtrar mensajes indeseados antes de enviarlos a su destinatario final. Los mensajes tienen una longitud máxima de 160 caracteres, por lo que el texto resulta poco para realizar la clasificación, en comparación con textos más largos (como los emails). Adicionalmente, los errores de digitación dificultan el proceso de detección automática.

## Descripción del problema en términos de los datos

Se tiene una muestra contiene 5574 mensajes en inglés, no codificados y clasificados como legítimos (ham) o spam (http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). La información está almacenada en el archivo `datos/spam-sms.zip`.El problema en términos de los datos consiste en clasificar si un mensaje SMS es legítico o spam, a partir del análisis de las palabras que contiente, partiendo del supuesto de que ciertas palabras que son más frecuentes dependiendo del tipo de mensaje. Esto implica que en la fase de preparación de los datos se deben extraer las palabras que contiene cada mensaje para poder realizar el análsis.

## Aproximaciones posibles

En este caso, se desea comparar los resultados de un modelo de redes neuronales artificiales y otras técnicas estadísticas para realizar la clasificación.

## Requerimientos

Usted debe:

* Preprocesar los datos para representarlos usando bag-of-words.


* Construir un modelo de regresión logística como punto base para la comparación con otros modelos más complejos.


* Construir un modelo de redes neuronales artificiales. Asimismo, debe determinar el número de neuronas en la capa o capas ocultas.


* Utiizar una técnica como crossvalidation u otra similar para establecer la robustez del modelo.


* Presentar métricas de desempeño para establecer las bondades y falencias de cada clasificador.

In [29]:
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf 
import matplotlib.pyplot as plt
import re
import nltk
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix as matriz
from sklearn.metrics import accuracy_score as accu
from tensorflow import keras
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

<p>Se leen los datos de el txt. Se crean dos columnas en el DataFrame Datos, una para ver si es ham o spam, y otra con el mensaje envidao. Luego se cambia la clasificacion de ham y spam a 1 y 0 para que luego este DataSet sirva para entrenar el modelo.</p>

In [2]:
Datos = pd.DataFrame (columns = ['H/S','Texto'])
with open('datos/SMSSpamCollection.txt', encoding='utf-8') as texto:
    Leer = texto.read().split("\n")
    for i in Leer[:-1]:
        Ham = "ham\t"
        Spam = "spam\t"
        CambioHam = "ham????????????????"
        CambioSpam = "spam????????????????"
        i = re.sub(Ham, CambioHam, i)
        i = re.sub(Spam, CambioSpam, i)
        i = i.split("????????????????")
        if (len(i) == 2):
            Datos = Datos.append({'H/S': i[0], 'Texto':i[1:]}, ignore_index=True)

In [3]:
Texto = pd.DataFrame (columns = ['Texto'])
for i in Datos['Texto']:
    Texto=Texto.append({'Texto': i[0].split()}, ignore_index=True)
    #print(i[0].split())

In [4]:
HS = Datos['H/S'].replace({'ham': 1, 'spam': 0}).astype('int')
stem = nltk.stem.SnowballStemmer('english', ignore_stopwords=True)
Palabras = stopwords.words("english")

PalabrasFinal = []
for i in Texto['Texto']:
    ST = [stem.stem(a) for a in i]
    PalabrasFinal.append(' '.join(ST))

PalabrasFinal = list(filter(None, PalabrasFinal))

<p><strong>Se crea el Bag of words que se va a utilizar para probar el modelo.</strong></p>

In [5]:
Vec = CountVectorizer(PalabrasFinal)
VexTex = Vec.fit_transform(PalabrasFinal)
VexTex= VexTex.toarray()

<p><strong>Entrenamiento y regresi&oacute;n lineal sobre los datos.</strong></p>

In [6]:
XEnt, XTest, YEnt, YTest = train_test_split(VexTex, HS, stratify=HS, test_size=0.2, random_state=123)

In [7]:
LR = LogisticRegression()
LR.fit(XEnt,YEnt)
Pred = LR.predict(XTest)

<p><strong>Matriz de confusi&oacute;n de la predicci&oacute;n de la regresi&oacute;n lineal.</strong></p>

In [38]:
print(matriz(YTest,Pred))

[[126  23]
 [  2 964]]


<p><strong>Accuracy de la predicci&oacute;n de la regresi&oacute;n lineal.</strong></p>

In [37]:
print(accu(YTest,Pred))

0.9775784753363229


<p><span style='color: rgb(0, 0, 0); font-family: "Helvetica Neue", Helvetica, Arial, sans-serif; font-size: 14px; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: 400; letter-spacing: normal; orphans: 2; text-align: justify; text-indent: 0px; text-transform: none; white-space: normal; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; background-color: rgb(255, 255, 255); text-decoration-style: initial; text-decoration-color: initial; display: inline !important; float: none;'>Peque&ntilde;a prueba para ver si esta corriendo el tensorflow o no, en versiones anteriores no esta corriendo por defecto.</span> </p>

In [9]:
if tf.executing_eagerly() == False:
    tf.enable_eager_execution()

<p><strong><span style="font-size: 
      20px;">Creaci&oacute;n de modelo de redes neuronales</span></strong></p>

<p>B&uacute;squeda de numero optimo de neuronas por capa.</p>

In [10]:
ResultadosN = [0,0]
for i in range(1,10):
    Modelo = keras.Sequential()

    Modelo.add(keras.layers.Dense(units = i, activation=tf.nn.relu, input_dim = XEnt.shape[1]))
    Modelo.add(keras.layers.Dense(units = 1, activation=tf.nn.sigmoid))
    
    Modelo.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc','mse'])
    
    Modelo.fit(XEnt,YEnt,epochs=50,batch_size=30, verbose=0)
    a = Modelo.evaluate(XTest, YTest)
    if(a[1]>ResultadosN[1]):
        ResultadosN = [i,a[1]]
    Modelo = 0

1115/1115 [==============================] - 0s 429us/sample - loss: 0.0968 - acc: 0.9839 - mse: 0.0148


<p>B&uacute;squeda de numero optimo de capas.</p>

In [11]:
ResultadosC = [0,0]
for i in range(1,10): 
    Modelo = keras.Sequential()
    
    for j in range (0,i):
        Modelo.add(keras.layers.Dense(units = ResultadosN[0], activation=tf.nn.relu, input_dim = XEnt.shape[1]))
        
    Modelo.add(keras.layers.Dense(units = 1, activation=tf.nn.sigmoid))
    
    Modelo.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc','mse'])
    
    Modelo.fit(XEnt,YEnt,epochs=50,batch_size=30, verbose=0)
    a = Modelo.evaluate(XTest, YTest)
    if(a[1]>ResultadosC[1]):
        ResultadosC = [i,a[1]]
    Modelo = 0

1115/1115 [==============================] - 1s 520us/sample - loss: 0.3932 - acc: 0.8664 - mse: 0.1158


In [12]:
print(ResultadosN, ResultadosC)

[1, 0.99013454] [1, 0.99013454]


<p>Modelo con numero de capas y neuronas optimo.</p>

In [13]:
Modelo = keras.Sequential()

for j in range(0,ResultadosC[0]):
    Modelo.add(keras.layers.Dense(units =ResultadosN[0], activation=tf.nn.relu, input_dim = XEnt.shape[1]))
    
Modelo.add(keras.layers.Dense(units = 1, activation=tf.nn.sigmoid))
    
Modelo.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc','mse'])

Modelo.fit(XEnt,YEnt,epochs=100,batch_size=30,verbose=0)
a = Modelo.evaluate(XTest, YTest)

1115/1115 [==============================] - 0s 415us/sample - loss: 0.0465 - acc: 0.9874 - mse: 0.0110


<p><strong>Prueba con Cross Validation para el modelo &oacute;ptimo encontrado antes.</strong></p>

In [24]:
def validacion(Indice, IndiceTest, Variables, Resultados):
    XEnt = Variables[Indice]
    XTest = Variables[IndiceTest]
    YEnt = Resultados.iloc[Indice]
    YTest = Resultados.iloc[IndiceTest]

    Modelo = keras.Sequential()

    for j in range(0,ResultadosC[0]):
        Modelo.add(keras.layers.Dense(units =ResultadosN[0], activation=tf.nn.relu, input_dim = XEnt.shape[1]))

    Modelo.add(keras.layers.Dense(units = 1, activation=tf.nn.sigmoid))

    Modelo.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc','mse'])

    Modelo.fit(XEnt,YEnt,epochs=100,batch_size=30,verbose=0)
    a = Modelo.evaluate(XTest, YTest)
    
    return a

K = KFold(n_splits=4)

predictions = []

for IndiceEnt, IndiceTest in K.split(XEnt):
    predictions.append(validacion(IndiceEnt,IndiceTest,XEnt,YEnt))

1114/1114 [==============================] - 0s 424us/sample - loss: 0.1940 - acc: 0.9713 - mse: 0.0239


NameError: name 'perdictions' is not defined

In [34]:
b = []
c = []
for i in predictions:
    b.append(i[2])
    c.append(i[1])
print(sum(b)/len(b))
print(sum(c)/len(c))

0.019033532589673996
0.9777962565422058


<p><span style="font-size: 
      22px;"><strong>Conclusiones</strong></span></p>

<p><strong><span style="font-size: 
      18px;">Accuracy</span></strong><br><br>- Regresi&oacute;n lineal: 0.9775784753363229</p>
<p>- Red neuronal: 0.9777962565422058</p>
